In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
test = pd.read_csv('./air_quality/PRSA_Data_Dongsi_20130301-20170228.csv')
test['datetime'] = pd.to_datetime(test[['year', 'month', 'day', 'hour']])
# test.head()
test['datetime'].isna().sum()

0